# Visualize Query Results

Jenna Jordan

23 January 2020 - 11 February 2020

### Purpose

This interactive visualization tool enables the researchers to explore the monthly time-series data across queries and publishers, to visually inspect for trends and decide which queries to use in the final analysis.

In [1]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
import plotly.express as px
from IPython.display import display
from IPython.display import clear_output

In [2]:
df_bln_6pubs = pd.read_csv("../Data/Visualize/bln-queries_6pubs_26Feb.csv", parse_dates=['publication_date'], dtype={'publisher':'category'})

## Queries - BulkLexisNexis, 1977-2019

#### BLN_total

```
(content:*) 
AND source_name:BulkLexisNexis
```

#### insect_population

```
(content: 
    (insect OR pollinator OR bee OR honeybee OR moth) 
    AND ("insect population"~5 OR "pollinator population"~5 OR "bee population"~5 OR "honeybee population"~5 OR "moth population"~5 OR "biological diversity" OR biodiversity OR biomass OR ecolog* OR ecosystem* OR entomolog*) 
    AND (study OR professor OR experiment OR research OR analysis OR data)
 ) 
AND (source_name:BulkLexisNexis)

```

#### insect_decline

```
(content: 
    (insect OR pollinator OR bee OR honeybee OR moth) 
    AND ("insect population"~5 OR "pollinator population"~5 OR "bee population"~5 OR "honeybee population"~5 OR "moth population"~5 OR "biological diversity" OR biodiversity OR biomass OR ecolog* OR ecosystem* OR entomolog*) 
    AND (study OR professor OR experiment OR research OR analysis OR data) 
    AND (crisis OR "colony collapse" OR apocalypse OR armageddon OR extinct OR "insect decline"~5 OR "insect drop"~5 OR "insect decrease"~5 OR "insect disappear"~5 OR "population decline"~5 OR "population drop"~5 OR "population decrease"~5 OR "population disappear"~5 OR "abundance decline"~5 OR "abundance drop"~5 OR "abundance decrease"~5 OR "abundance disappear"~5)
) 
AND (source_name:BulkLexisNexis)

```

#### pollinator_population

```
(content: 
    ((insect AND pollinator) OR (bee OR honeybee OR moth)) 
    AND ("insect population"~5 OR "pollinator population"~5 OR "bee population"~5 OR "honeybee population"~5 OR "moth population"~5 OR "biological diversity" OR biodiversity OR biomass OR ecolog* OR ecosystem* OR entomolog*) 
    AND (study OR professor OR experiment OR research OR analysis OR data)
) 
AND (source_name:BulkLexisNexis)

```

#### pollinator_decline

```
(content: 
    ((insect AND pollinator) OR (bee OR honeybee OR moth)) 
    AND ("insect population"~5 OR "pollinator population"~5 OR "bee population"~5 OR "honeybee population"~5 OR "moth population"~5 OR "biological diversity" OR biodiversity OR biomass OR ecolog* OR ecosystem* OR entomolog*) 
    AND (study OR professor OR experiment OR research OR analysis OR data) 
    AND (crisis OR "colony collapse" OR apocalypse OR armageddon OR extinct OR "insect decline"~5 OR "insect drop"~5 OR "insect decrease"~5 OR "insect disappear"~5 OR "population decline"~5 OR "population drop"~5 OR "population decrease"~5 OR "population disappear"~5 OR "abundance decline"~5 OR "abundance drop"~5 OR "abundance decrease"~5 OR "abundance disappear"~5)
) 
AND (source_name:BulkLexisNexis)

```

#### insect_apocalypse

```
(content:"insect apocalypse"~5 OR "insect armageddon"~5 OR "beepocalypse") 
AND (source_name:BulkLexisNexis)
```

#### colony_collapse

```
(content:"colony collapse" AND (bee OR honeybee)) 
AND (source_name:BulkLexisNexis)
```

#### climate_change

```
(content:"climate change" OR "global warming") 
AND (source_name:BulkLexisNexis)
```

#### climate_change_IPCCreport

```
(content:
    ("climate change" OR "global warming") 
    AND ("IPCC" OR "Intergovernmental Panel on Climate Change") 
    AND report
) 
AND (source_name:BulkLexisNexis)
```

#### insect_population_studies

```
(content: 
    ("Krefeld" OR "the German study" OR "Hans de Kroon" OR "Martin Sorg" OR "Werner Stenmans" OR "Dave Goulson" OR "Brad Lister" OR "Andres Garcia" OR "the Puerto Rico study" OR "S?nchez-Bayo" OR "Wyckhuys" OR "Rob Dunn" OR "David Wagner" OR "Chris Thomas" OR "Anders Tottrup" OR "Kevin Gaston" OR "Chris Thomas" OR "Roel van Klink" OR "Arthur Shapiro" OR "Aletta Bonn" OR "E.O. Wilson") 
    AND (insect OR pollinator OR bee OR honeybee OR moth) 
    AND ("insect population"~5 OR "pollinator population"~5 OR "bee population"~5 OR "honeybee population"~5 OR "moth population"~5 OR "biological diversity" OR biodiversity OR biomass OR ecolog* OR ecosystem* OR entomolog*) 
    AND (study OR professor OR experiment OR research OR analysis OR data)
) 
AND (source_name:BulkLexisNexis)
```

In [3]:
publisher_options = [('New York Times', 'NYT'), ('Washington Post', 'WP'), ('Associated Press', 'AP'), 
                     ('Agence France Presse', 'AFP'), ('Xinhua General News Service', 'XGNS'), 
                     ('Deutsche Presse-Agentur', 'DPA')]
query_options = list(df_bln_6pubs['query'].unique())

In [4]:
choose_CountOrProp = widgets.ToggleButtons(
    options=[('Count', 'count'), ('Proportion', 'proportion')],
    description='Article: ', layout=widgets.Layout(width='25%')
)

In [5]:
choose_comparison = widgets.ToggleButtons(
    options=[('Queries', 'publisher'), ('Publishers', 'query')],
    description='Compare: ', layout=widgets.Layout(width='25%')
)

In [6]:
choose_filter = widgets.Dropdown(options=publisher_options, description='Choose: ', layout=widgets.Layout(width='25%'))

In [7]:
plot_button = widgets.Button(description="Plot")

In [8]:
clear_button = widgets.Button(description="Clear plots")

In [10]:
# set up control box organization
buttons = widgets.VBox(children=[plot_button, clear_button], layout=widgets.Layout(width='25%'))
graph_controls = widgets.HBox(children=[choose_CountOrProp, choose_comparison, choose_filter, buttons])

# if "Compare" changes, change the options for "Choose"
def change_graph_controls(change):
    graph_comparison_type = change['new']
    if graph_comparison_type == 'publisher':
        choose_filter.options = publisher_options
    elif graph_comparison_type == 'query':
        choose_filter.options = query_options

choose_comparison.observe(change_graph_controls, ['value'])

# function to create the plot
def create_plot(query_column = choose_comparison.value, query_value = choose_filter.value, y_axis = choose_CountOrProp.value):
    if query_column  == 'publisher':
        color = 'query'
    elif query_column  == 'query':
        color = 'publisher'
    
    query_text = query_column + "=='" + query_value + "'"
    title_text = query_column + ": " + query_value
    
    fig = px.line(df_bln_6pubs.query(query_text), x="publication_date", y=y_axis, color=color, title=title_text)
    
    fig.update_layout(xaxis_rangeslider_visible=True)
    
    return fig

out = widgets.Output()
with out:
    display(graph_controls)

# when "add chart" button is clicked, add new chart below
def add_plot(b):
    with out:
        fig = create_plot(query_column = choose_comparison.value, query_value = choose_filter.value, y_axis = choose_CountOrProp.value)
        display(fig)

plot_button.on_click(add_plot)

# when "new plot" button is clicked, clear all plots and generate a new plot
def clear_plots(b):
    with out:
        out.clear_output()
        display(graph_controls)
    
clear_button.on_click(clear_plots)

In [11]:
display(out)

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': "HBox(children=(ToggleButtons(descripti…